In [1]:
import os
import pandas as pd
import numpy as np
from scipy.io import loadmat


cwd = os.getcwd()
cwd = os.path.join(cwd,'ProcessedData')

data = {}

folders = os.listdir(cwd)
for fruit in folders:
    records = []
    for record in os.listdir(os.path.join(cwd,fruit)):
        records.append(loadmat(os.path.join(cwd,fruit,record)))
    data[fruit] = records

In [2]:
print(data['Banana'][0]['vals'][:1])

[[[-1.2633322e-01 -1.8298553e-01  7.8900003e-01  2.5500000e+02
    2.5500000e+02  2.5500000e+02]
  [-1.2633322e-01 -1.8298553e-01  7.8900003e-01  2.5500000e+02
    2.5500000e+02  2.5500000e+02]
  [-1.2403622e-01 -1.8276133e-01  7.8800005e-01  2.5500000e+02
    2.5500000e+02  2.5500000e+02]
  ...
  [ 2.3975252e-01 -1.7771065e-01  7.6800001e-01  2.5500000e+02
    2.5500000e+02  2.5500000e+02]
  [ 2.3975252e-01 -1.7771065e-01  7.6800001e-01  2.5500000e+02
    2.5500000e+02  2.5500000e+02]
  [ 2.3975252e-01 -1.7771065e-01  7.6800001e-01  2.5500000e+02
    2.5500000e+02  2.5500000e+02]]]


In [3]:
X,y = [],[]
for fruit_name in data:
    for fruit_sample in data[fruit]:
        X.append(fruit_sample['vals'])
        y.append(fruit_name)

In [4]:
y = np.asarray(y)
X = np.asarray(X)
print(y.shape)
print(X.shape)

(500,)
(500, 500, 500, 6)


In [5]:
X_new = X[:,:,:,[3,4,5,0,1,2]]

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder  

le = LabelEncoder()
y = le.fit_transform(y)

y = to_categorical(y)

In [7]:
X_new[:,:,:,3:5] += 1
X_new[:,:,:,:3] /= 255
X_new[:,:,:,3] /= np.amax(X_new[:,:,:,3])
X_new[:,:,:,4] /= np.amax(X_new[:,:,:,4])
X_new[:,:,:,5] /= np.amax(X_new[:,:,:,5])

In [8]:
from sklearn.model_selection import train_test_split as split

X_train, X_test, y_train, y_test = split(X_new, y, test_size=.3)

In [9]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

350
150
350
150


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D


model = Sequential()
model.add(Conv2D(100, (5, 5), activation='relu', input_shape=(500,500,6)))
model.add(MaxPooling2D(pool_size=(5, 5)))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

In [11]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 496, 496, 100)     15100     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 980100)            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 4900505   
Total params: 4,915,605
Trainable params: 4,915,605
Non-trainable params: 0
_________________________________________________________________


In [13]:
X_train, X_val, y_train, y_val = split(X_train, y_train, test_size=0.3)
print(len(X_train))
print(len(X_val))
print(len(y_train))
print(len(y_val))

245
105
245
105


In [ ]:
hist = model.fit(X_train, y_train, batch_size=256, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
